In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

In [4]:
from operator import itemgetter

from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

model = AzureChatOpenAI(temperature=0.3,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    deployment_name="gpt-35-turbo",
    openai_api_key=openai.api_key,
    openai_api_type = openai.api_type,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

In [3]:
memory = ConversationBufferMemory(return_messages=True)
memory.load_memory_variables({})

{'history': []}

In [5]:
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | model
    | StrOutputParser()
)

In [7]:
inputs = {"input": "hi im Shiva"}
response = chain.invoke(inputs)
response

'Hello Shiva! How can I assist you today?'

In [8]:
memory.save_context(inputs, {"output": response})

In [9]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi im Shiva'),
  AIMessage(content='Hello Shiva! How can I assist you today?')]}

In [10]:
memory.load_memory_variables

<bound method ConversationBufferMemory.load_memory_variables of ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='hi im Shiva'), AIMessage(content='Hello Shiva! How can I assist you today?')]), return_messages=True)>

In [11]:
inputs = {"input": "whats my name"}
response = chain.invoke(inputs)
response

'Your name is Shiva.'